# Basic Statistics-2: Hospital Patient Data Analysis
This notebook follows the exact steps from the assignment to clean, merge, and enrich the patient/billing datasets.

## 1. Load datasets

In [1]:
# LOAD DATA
import pandas as pd
import numpy as np

patients = pd.read_csv(r"/mnt/data/Patient_Data (1).csv")
billing = pd.read_csv(r"/mnt/data/Billing_Data (1).csv")
patients.head(), billing.head()

(   PatientID     Name   Department     Doctor  BillAmount  ReceptionistID  \
 0        101    Alice   Cardiology  Dr. Smith      5000.0               1   
 1        102      Bob    Neurology   Dr. John         NaN               2   
 2        103  Charlie  Orthopedics    Dr. Lee      7500.0               1   
 3        104    David   Cardiology  Dr. Smith      6200.0               3   
 4        105      Eva  Dermatology   Dr. Rose         NaN               2   
 
         CheckInTime  
 0  2023-01-10 09:00  
 1  2023-01-11 10:30  
 2  2023-01-12 11:00  
 3  2023-01-13 12:00  
 4  2023-01-14 08:45  ,
    PatientID  InsuranceCovered  FinalAmount
 0        101              2000         3000
 1        102              1500         3500
 2        103              2500         5000
 3        104              3000         3200
 4        105              1000         4000)

## 2. Show summary with info()

In [2]:
# INFO() SUMMARY
info_patients = pd.DataFrame({
    "column": patients.columns,
    "dtype": [str(t) for t in patients.dtypes],
    "non_null": [int(patients[c].notna().sum()) for c in patients.columns]
})
info_patients

,column,dtype,non_null
0,PatientID,int64,6
1,Name,object,6
2,Department,object,6
3,Doctor,object,6
4,BillAmount,float64,4
5,ReceptionistID,int64,6
6,CheckInTime,object,6


## 3. Select only billing-relevant columns

In [3]:
def safe_cols(df, cols):
    return [c for c in cols if c in df.columns]

billing_cols = ['PatientID', 'Department', 'Doctor', 'BillAmount']
patients_billing_view = patients[safe_cols(patients, billing_cols)].copy()
patients_billing_view.head()

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


## 4. Drop administrative columns

In [4]:
admin_cols = ['ReceptionistID', 'CheckInTime']
patients_dropped = patients.drop(columns=safe_cols(patients, admin_cols), errors='ignore')
patients_dropped.head()

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


## 5. Groupby: Total bill per Department

In [5]:
bill_source_col = 'BillAmount' if 'BillAmount' in billing.columns else ('BillAmount' if 'BillAmount' in patients.columns else None)

if bill_source_col and 'Department' in billing.columns:
    groupby_df = billing.groupby('Department', dropna=False)[bill_source_col].sum().reset_index(name='TotalBillAmount')
elif bill_source_col and 'Department' in patients.columns:
    groupby_df = patients.groupby('Department', dropna=False)[bill_source_col].sum().reset_index(name='TotalBillAmount')
else:
    groupby_df = None

groupby_df

,Department,TotalBillAmount
0,Cardiology,16200.0
1,Dermatology,0.0
2,Neurology,0.0
3,Orthopedics,7500.0


## 6. Remove duplicate patients (PatientID)

In [6]:
if 'PatientID' in patients.columns:
    patients_nodup = patients.drop_duplicates(subset=['PatientID'])
else:
    patients_nodup = patients.copy()
patients_nodup.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


## 7. Fill missing BillAmount with mean

In [7]:
def fill_bill_mean(df, col='BillAmount'):
    if col in df.columns:
        mean_val = df[col].mean(skipna=True)
        df[col] = df[col].fillna(mean_val)
        return mean_val
    return None

mean_patients_bill = fill_bill_mean(patients_nodup)
mean_billing_bill = fill_bill_mean(billing)

mean_patients_bill, mean_billing_bill

/tmp/ipykernel_915/3798517642.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(mean_val)


(6233.333333333333, None)

## 8. Merge billing with patient data on PatientID

In [8]:
if 'PatientID' in patients_nodup.columns and 'PatientID' in billing.columns:
    merged = pd.merge(patients_nodup, billing, on='PatientID', how='left', suffixes=('_pat', '_bill'))
else:
    merged = patients_nodup.copy()
merged.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,1,2023-01-10 09:00,2000,3000
1,102,Bob,Neurology,Dr. John,6233.333333,2,2023-01-11 10:30,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,1,2023-01-12 11:00,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.000000,3,2023-01-13 12:00,3000,3200
4,105,Eva,Dermatology,Dr. Rose,6233.333333,2,2023-01-14 08:45,1000,4000


## 9. Concatenate new patients for current week (row-wise)

In [9]:
overlap_cols = merged.columns.tolist()

rows = []
for i, dep, doc, amt in [(1,'Cardiology','Dr. A',5500),(2,'Neurology','Dr. B',7200),(3,'Orthopedics','Dr. C',4800)]:
    row = {c: np.nan for c in overlap_cols}
    if 'PatientID' in row: row['PatientID'] = f"NEW{i:03d}"
    if 'Department' in row: row['Department'] = dep
    if 'Doctor' in row: row['Doctor'] = doc
    # choose the best BillAmount column present
    for col in ['BillAmount_bill','BillAmount_pat','BillAmount']:
        if col in row:
            row[col] = amt
            break
    rows.append(row)

new_patients_week = pd.DataFrame(rows, columns=overlap_cols)
concatenated_rows = pd.concat([merged, new_patients_week], ignore_index=True)
concatenated_rows.tail(6)

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime,InsuranceCovered,FinalAmount
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,1.0,2023-01-12 11:00,2500.0,5000.0
3,104,David,Cardiology,Dr. Smith,6200.000000,3.0,2023-01-13 12:00,3000.0,3200.0
4,105,Eva,Dermatology,Dr. Rose,6233.333333,2.0,2023-01-14 08:45,1000.0,4000.0
5,NEW001,NaN,Cardiology,Dr. A,5500.000000,NaN,NaN,NaN,NaN
6,NEW002,NaN,Neurology,Dr. B,7200.000000,NaN,NaN,NaN,NaN
7,NEW003,NaN,Orthopedics,Dr. C,4800.000000,NaN,NaN,NaN,NaN


## 10. Concatenate new billing category columns (column-wise)

In [10]:
result = concatenated_rows.copy()

bill_for_final = None
for c in ['BillAmount_bill','BillAmount_pat','BillAmount']:
    if c in result.columns:
        bill_for_final = c
        break

if 'InsuranceCovered' not in result.columns:
    np.random.seed(42)
    result['InsuranceCovered'] = np.random.choice([True, False], size=len(result))

if bill_for_final:
    result[bill_for_final] = pd.to_numeric(result[bill_for_final], errors='coerce')
    fill_mean = result[bill_for_final].mean(skipna=True)
    result[bill_for_final] = result[bill_for_final].fillna(fill_mean)
    result['FinalAmount'] = np.where(result['InsuranceCovered'],
                                     result[bill_for_final] * 0.80,   # 20% covered
                                     result[bill_for_final])
else:
    result['FinalAmount'] = np.nan

result.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,1.0,2023-01-10 09:00,2000.0,4000.000000
1,102,Bob,Neurology,Dr. John,6233.333333,2.0,2023-01-11 10:30,1500.0,4986.666667
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,1.0,2023-01-12 11:00,2500.0,6000.000000
3,104,David,Cardiology,Dr. Smith,6200.000000,3.0,2023-01-13 12:00,3000.0,4960.000000
4,105,Eva,Dermatology,Dr. Rose,6233.333333,2.0,2023-01-14 08:45,1000.0,4986.666667


## 11. Final cleaned dataset

In [11]:
result.head(20)

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,1.0,2023-01-10 09:00,2000.0,4000.000000
1,102,Bob,Neurology,Dr. John,6233.333333,2.0,2023-01-11 10:30,1500.0,4986.666667
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,1.0,2023-01-12 11:00,2500.0,6000.000000
3,104,David,Cardiology,Dr. Smith,6200.000000,3.0,2023-01-13 12:00,3000.0,4960.000000
4,105,Eva,Dermatology,Dr. Rose,6233.333333,2.0,2023-01-14 08:45,1000.0,4986.666667
5,NEW001,NaN,Cardiology,Dr. A,5500.000000,NaN,NaN,NaN,4400.000000
6,NEW002,NaN,Neurology,Dr. B,7200.000000,NaN,NaN,NaN,5760.000000
7,NEW003,NaN,Orthopedics,Dr. C,4800.000000,NaN,NaN,NaN,3840.000000


### Saved outputs

In [12]:
result.to_csv(r"/mnt/data/Final_Cleaned_Billing_Dataset.csv", index=False)
"/mnt/data/Final_Cleaned_Billing_Dataset.csv" 

/home/sandbox/.local/lib/python3.11/site-packages/pandas/core/internals/blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


'/mnt/data/Final_Cleaned_Billing_Dataset.csv'